# Inverse Reachability Repositioning

## 1. Offline

### IRM Data

- Sample: ./config/robocare_right_irm.npy
  - This sample has 7899 points.
  - The supported z of target is {0.7, 0.75, 0.8}.

| Index | Name                     | Unit       | Remark           |
| ----- | ------------------------ | ---------- | ---------------- |
| 0     | Mobile Base x            | meter      | For query output |
| 1     | Mobile Base y            | meter      | For query output |
| 2     | Target Object z (height) | meter      | For query input  |
| 3     | EEP x                    | meter      | For IK solver    |
| 4     | EEP y                    | meter      | For IK solver    |
| 5     | EEP z                    | meter      | For IK solver    |
| 6     | EE Roll                  | **DEGREE** |                  |
| 7     | EE Pitch                 | **DEGREE** |                  |
| 8     | EE Yaw                   | **DEGREE** | Cr               |
| 9     | Manipulability           | -          |                  |
| 10    | Joint_0 value            | radian     |                  |
| 11    | Joint_1 value            | radian     |                  |
| >=12  | Joint_2... values        | radian     |                  |

### Configuration

In [ ]:
base_radius = 0.3
irm_path = "../config/robocare_right_irm.npy"

### Initialization

In [2]:
import numpy as np
import irm

reposition = irm.InverseReachabilityMap(irm_path, base_radius, is_jupyter=True)

## 2. Online

In [3]:
from collision import CollisionBox

Pt = (0.0, 0.0)
Obs = [CollisionBox((0.2, 0.0), np.radians(0.0), 0.4, 1.0)]
# Cr = (30, 30)  # min, max
Cr = (0, 0)
Ct = (-np.radians(180), np.radians(180)) # min, max

In [6]:
reposition.calc(Pt, Obs, Cr, Ct, (0.05, 1.0, 0.02))

Cut the range of `Cr` from `Fraw` and set it to `Fcut`.
min Cr:  0.0
max Cr:  0.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

And extract only the maximum as a `Fmax`.
sq_sections.shape:  (48,)
Fmax.shape:  (26, 18)
When Ct == 0.0 deg


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Wipe the `Fmax` in the range of `Ct`. => `Fwiped`
min Ct:  -180.0
max Ct:  180.0
Fwiped.shape:  (26, 20)
Interval is  0.02  [m].


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Remove all obstacle areas from `Fwiped` with the offset of `Rsize`. => `Fclean`
Fclean.shape:  (26, 20)
Fclean.shape:  (0, 20)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Find a Feasible Mobile Manipulation (from Real Data)

In [5]:
%matplotlib widget

import numpy as np
import feasibility_map as fmap

ModuleNotFoundError: No module named 'feasibility_map'

## 2. Online

### 2.1. Input

Known
1. `self.feasi_layers`: feasibility map
2. `self.robot_radius`: robot base (circle radius)

---
Input

1. `Pt`: Position of the target object (relative to the robot’s current pose) => `(x, y)`
2. `Obs`: Area list of ground obstacles => `[CollisionModel, ...]`
3. `Cr`: Constraints on the approach angle (relative to the robot heading) (-90 ~ 90) => `(min, max)`
4. `Ct`: Constraints on the approach angle (relative to the target heading) (-180 ~ 180) => `(min, max)`

In [ ]:
from collision import CollisionBox

Pt = (0.0, 0.0)
Obs = [CollisionBox((0.2, 0.0), np.radians(0.0), 0.45, 1.0)]
# Cr = (30, 30)  # min, max
Cr = (0, 0)
Ct = (-np.radians(180), np.radians(180)) # min, max

### 2.2. Process

1. Cut the range of `Cr` from `F` and set it to `Fcut`.
2. ~~[SKIP] Scan the maximum points for the radius and angle by each circle in `Fcut`.~~
3. And extract only the maximum as a `Fmax`.
4. Wipe the `Fmax` in the range of `Ct`.
5. Remove all obstacle areas from `Fwiped` with the offset of `Rsize`.

In [ ]:
F.calc(Pt, Obs, Cr, Ct, (0.05, 0.5, 0.02))

### 2.3. Output

1. Candidate poses (sorted in descending order of manipulability)

In [ ]:
candidates = F.get_candidates()